# Name Screening

## Data Analysis

## Table of Contents <a class="anchor" id="toc"></a>

1. [Function Definitions](#func-defs)
    1. [Name Screening Solutions](#first-func-def)
    2. [Plot - Execution Speed](#second-func-def)
    3. [Plot - Number of matches & Execution Time](#third-func-def)
    4. [Plot - Accuracy](#fourth-func-def)
    5. [Metric Comparision for final distance metric](#fifth-func-def)
    6. [Threshold Analysis for finalizing thresholds](#sixth-func-def)
2. [Load data](#load-data)
3. [Data Analysis](#data-analysis)
    1. [Generate test data](#gen-test)
    2. [Find optimal thresholds](#find-thresholds)
        1. [Finding metrics](#find-thresholds-part1)
        2. [Finding Lower Limit of thresholds](#find-thresholds-part2)
        3. [Finding Upper Limit of thresholds](#find-thresholds-part3)
    3. [Finding Optimal Solution](#find-solution)
4. [Generate performance plots](#gen-plots)
    1. [Using True positives](#sec_4_1)
    2. [Using False positives](#sec_4_2)
5. [Compare performance with name list without the randomly generated names](#alt-find-solution)
    1. [Loading data source and test set](#sec_5_1)
    2. [Finding best solution using true positive](#sec_5_2)
    3. [Finding best solution using false positive](#sec_5_3)
    4. [Generate performance plots](#sec_5_4)

## Libraries

In [ ]:
from platform import python_version
print("Python Version:", python_version())

import matplotlib
print("Matplotlib Version:", matplotlib. __version__) 

import warnings
#warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')

# pip install abydos
# pip install python-Levenshtein


import re
import os
import time

import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from abydos import phonetic, distance
from Levenshtein import ratio as lev_ratio
from Levenshtein import seqratio as lev_seqratio
from Levenshtein import setratio as lev_setratio

# 1. Function Definitions <a class="anchor" id="func-defs"></a>

Go to [Table of Contents](#toc)

## 1.1. Function Definition - All name screening solutions <a class="anchor" id="first-func-def"></a>

Go to [Table of Contents](#toc)

The function provided **FIVE** options. They are as follows:
1. Applying Levenshtein set ratio on the names directly ***(Traditional approach)***
2. Applying Levenshtein set ratio on the phonemes of the names
3. Using custom Levenshtein ratio to measure similarity between the phonemes of the names
4. Using BOTH Levenshtein set ratio on names AND custom Levenshtein ratio on phonemes of the names for comparision ***(Proposed approach 1)***
5. Using EITHER Levenshtein set ratio on names OR custom Levenshtein ratio on phonemes of the names for comparision ***(Proposed approach 2)***

### Find Ideal model using true positive records

In [ ]:
def solutions(sol_type, db, func, thres, name, actual_name, enable_prints=True):
    
    print()
    sol_name = ""
    if sol_type == 1:
        print('Solution 1 - Levenshtein set ratio on name matching')
        sol_name = "Baseline Solution"
    elif sol_type == 2:
        print('Solution 2 - Levenshtein set ratio on Phoneme matching')
        sol_name = "Intermediate Solution 1"
    elif sol_type == 3:
        print('Solution 3 - Custom Levenshtein Ratio on Phonemes')
        sol_name = "Intermediate Solution 2"
    elif sol_type == 4:
        print('Solution 4 - Levenshtein set ratio AND Custom Levenshtein Ratio on Phonemes')
        sol_name = "Proposed Solution 1"
    elif sol_type == 5:
        print('Solution 5 - Levenshtein set ratio OR Custom Levenshtein Ratio on Phonemes')
        sol_name = "Proposed Solution 2"
    else:
        print('Invalid Option! Choose from 1 to 5')
        return None
    
    print('Searched Name:', name, '\nActual Name:', actual_name)
    
    results = pd.DataFrame()
    
    start_time= time.time()
    
    if sol_type != 1:
        pn = []
        for t in name.split():
            pn.append(phonetic.DoubleMetaphone().encode(t)[0])
            
        if enable_prints:
            print(f"Phoneme for {name} is : {pn}")
    
    for row in db.iterrows():
        
        if sol_type == 1:
            metric = func(row[1]['Name'].lower(), name.lower())
        elif sol_type == 2:
            metric = func(' '.join(row[1]['Phonemes']).lower(), ' '.join(pn).lower())
        elif sol_type == 3:
            dist_score = []
            dist_score = [max([(func(i,j)) for j in row[1]['Phonemes']]) for i in pn]
            metric = np.mean(dist_score)
        elif sol_type == 4 or sol_type == 5:
            metric1 = func[0](row[1]['Name'].lower(), name.lower())
            dist_score = []
            dist_score = [max([(func[1](i,j)) for j in row[1]['Phonemes']]) for i in pn]
            metric2 = np.mean(dist_score)
        
        if sol_type == 1 or sol_type == 2 or sol_type == 3:
            condition = (metric >= thres)
        elif sol_type == 4:
            condition = (metric1 >= thres[0] and metric2 >= thres[1])
        elif sol_type == 5:
            condition = (metric1 >= thres[0] or metric2 >= thres[1])

        
        if condition:
            
            if sol_type == 1:
                dist = np.round(lev_ratio(row[1]['Name'].lower(), name.lower()), 2)
            elif sol_type == 2:
                dist = np.round(lev_ratio(' '.join(row[1]['Phonemes']).lower(), ' '.join(pn).lower()), 2)
            
            if sol_type == 1 or sol_type == 2:
                df2 = {'Name': row[1]['Name'], 
                       'LevSetRatio': metric, 
                       'Distance': dist}
            elif sol_type == 3:
                df2 = {'Name': row[1]['Name'], 
                       'Phoneme': row[1]['Phonemes'], 
                       'LevRatio': metric}
            elif sol_type == 4 or sol_type == 5:
                df2 = {'Name': row[1]['Name'], 
                       'Phoneme': row[1]['Phonemes'], 
                       'LevSetRatio': metric1, 
                       'LevRatio': metric2}

            results = results.append(df2, ignore_index = True)

    fin_time = np.round((time.time() - start_time), 2)
    print(f"--- Execution Time: {fin_time:,} seconds ---")
    
    if sol_type == 1 or sol_type == 2:
        if not results.empty:
            results.sort_values('LevSetRatio', ascending=False, inplace=True)
    elif sol_type == 3:
        if not results.empty:
            results.sort_values('LevRatio', ascending=False, inplace=True)
    elif sol_type == 4 or sol_type == 5:
        if not results.empty:
            results.sort_values(['LevRatio', 'LevSetRatio'], ascending=False, inplace=True)
    
    results.reset_index(drop=True, inplace=True)
    if enable_prints:
        print("Number of matched names:", results.shape[0])
    
    actual_present = False
    if not results.empty:
        if results[results.Name.str.contains(actual_name)].shape[0]:
            actual_present = True
            if enable_prints:
                print('Results contain the actual name!')
                display(results[results.Name.str.contains(actual_name)])
        else:
            actual_present = False
            if enable_prints:
                print('Results do not contain the actual name...')

    return sol_name, results, actual_present, fin_time



### Find Ideal model using false positive records

In [ ]:
def solutions2(sol_type, db, func, thres, name, enable_prints=True):
    
    print()
    sol_name = ""
    if sol_type == 1:
        print('Solution 1 - Levenshtein set ratio on name matching')
        sol_name = "Baseline Solution"
    elif sol_type == 2:
        print('Solution 2 - Levenshtein set ratio on Phoneme matching')
        sol_name = "Intermediate Solution 1"
    elif sol_type == 3:
        print('Solution 3 - Custom Levenshtein Ratio on Phonemes')
        sol_name = "Intermediate Solution 2"
    elif sol_type == 4:
        print('Solution 4 - Levenshtein set ratio AND Custom Levenshtein Ratio on Phonemes')
        sol_name = "Proposed Solution 1"
    elif sol_type == 5:
        print('Solution 5 - Levenshtein set ratio OR Custom Levenshtein Ratio on Phonemes')
        sol_name = "Proposed Solution 2"
    else:
        print('Invalid Option! Choose from 1 to 5')
        return None
    
    print('Searched Name:', name)
    
    results = pd.DataFrame()
    
    start_time= time.time()
    
    if sol_type != 1:
        pn = []
        for t in name.split():
            pn.append(phonetic.DoubleMetaphone().encode(t)[0])
            
        if enable_prints:
            print(f"Phoneme for {name} is : {pn}")
    
    for row in db.iterrows():
        
        if sol_type == 1:
            metric = func(row[1]['Name'].lower(), name.lower())
        elif sol_type == 2:
            metric = func(' '.join(row[1]['Phonemes']).lower(), ' '.join(pn).lower())
        elif sol_type == 3:
            dist_score = []
            dist_score = [max([(func(i,j)) for j in row[1]['Phonemes']]) for i in pn]
            metric = np.mean(dist_score)
        elif sol_type == 4 or sol_type == 5:
            metric1 = func[0](row[1]['Name'].lower(), name.lower())
            dist_score = []
            dist_score = [max([(func[1](i,j)) for j in row[1]['Phonemes']]) for i in pn]
            metric2 = np.mean(dist_score)
        
        if sol_type == 1 or sol_type == 2 or sol_type == 3:
            condition = (metric >= thres)
        elif sol_type == 4:
            condition = (metric1 >= thres[0] and metric2 >= thres[1])
        elif sol_type == 5:
            condition = (metric1 >= thres[0] or metric2 >= thres[1])

        
        if condition:
            
            if sol_type == 1:
                dist = np.round(lev_ratio(row[1]['Name'].lower(), name.lower()), 2)
            elif sol_type == 2:
                dist = np.round(lev_ratio(' '.join(row[1]['Phonemes']).lower(), ' '.join(pn).lower()), 2)
            
            if sol_type == 1 or sol_type == 2:
                df2 = {'Name': row[1]['Name'], 
                       'LevSetRatio': metric, 
                       'Distance': dist}
            elif sol_type == 3:
                df2 = {'Name': row[1]['Name'], 
                       'Phoneme': row[1]['Phonemes'], 
                       'LevRatio': metric}
            elif sol_type == 4 or sol_type == 5:
                df2 = {'Name': row[1]['Name'], 
                       'Phoneme': row[1]['Phonemes'], 
                       'LevSetRatio': metric1, 
                       'LevRatio': metric2}

            results = results.append(df2, ignore_index = True)

    fin_time = np.round((time.time() - start_time), 2)
    print(f"--- Execution Time: {fin_time:,} seconds ---")
    
    if sol_type == 1 or sol_type == 2:
        if not results.empty:
            results.sort_values('LevSetRatio', ascending=False, inplace=True)
    elif sol_type == 3:
        if not results.empty:
            results.sort_values('LevRatio', ascending=False, inplace=True)
    elif sol_type == 4 or sol_type == 5:
        if not results.empty:
            results.sort_values(['LevRatio', 'LevSetRatio'], ascending=False, inplace=True)
    
    results.reset_index(drop=True, inplace=True)
    if enable_prints:
        print("Number of matched names:", results.shape[0])


    return sol_name, results, fin_time

## 1.2. Function Definition - Plot horizonal bar for comparing execution speed <a class="anchor" id="second-func-def"></a>

Go to [Table of Contents](#toc)

In [ ]:
def show_exec_speed(results, searched_name, actual_name):
    fig, ax = plt.subplots(figsize=(10,3))

    sols = [val[0] for val in results]
    times = [val[3] for val in results]
    perc_change = [" "+str(np.round((val-times[0])*100/times[0], 2))+" %" for val in times]
    bar_colors = ['green' if val[2] else 'red' for val in results]


    ax.barh(sols, times, color=bar_colors)

    for index, val in enumerate(times):
        plt.text(val, index, val, 
                 color = 'black', horizontalalignment='left', verticalalignment='center')
        if index == 0:
            continue
        plt.text(val, index, perc_change[index], 
                 color = 'white', horizontalalignment='right', verticalalignment='center')

    ax.set_yticklabels(sols)

    # Labels are inverted by default
    ax.invert_yaxis()  

    ax.set_xlabel('Fetch time (seconds)')
    ax.set_title(f'Execution speed while searching for "{actual_name}" using "{searched_name}" ')
    plt.grid(True, axis = 'x')


    green_patch = mpatches.Patch(color='green', label='Name matched')
    red_patch = mpatches.Patch(color='red', label='Name not matched')
    ax.legend(handles=[green_patch, red_patch])

    plt.show()

    return None


## 1.3. Function Definition - Plot merged bar and line plot to compare number of false positive matches <a class="anchor" id="third-func-def"></a>

Go to [Table of Contents](#toc)

In [ ]:
def show_performance_plots(df, focal_entity , limits, xlabels, figsize):
    ax1 = sns.set_style(style=None, rc=None )

    fig, ax1 = plt.subplots(figsize=figsize)

    g = sns.lineplot(data = df['Time'], marker='o', sort = False, ax=ax1, label="Execution Time")
    g.set(ylim=limits[0])
    ax2 = ax1.twinx()
    for index, row in df.iterrows():
        g.text(row.name, 
                   row['Time'], 
                   np.round(row['Time'], 2), 
                   color='black', 
                   ha="center", 
                   size='large', 
                   verticalalignment='bottom')

    bplot = sns.barplot(data = df, x=focal_entity, y='Num_of_matches', alpha=0.5, ax=ax2, label="Number of matches")
    bplot.set(ylim=limits[1])
    for index, row in df.iterrows():
        bplot.text(row.name, 
                   row['Num_of_matches'], 
                   int(row['Num_of_matches']), 
                   color='black', 
                   ha="center", 
                   size='large', 
                   verticalalignment='baseline')

    #ax1.set_xticklabels(df[focal_entity], rotation=90)
    #ax1.set_xlabel(xlabels)
    ax1.set_xlabel('Solutions')
    ax1.set_ylabel('Execution Time (seconds)')
    ax2.set_ylabel('Number of matches')
    plt.title('Comparing Number of matches and Execution speed among different solutions')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.show()
    
    return None

## 1.4. Function Definition - Plot bar plot of accuracy of each model <a class="anchor" id="fourth-func-def"></a>

Go to [Table of Contents](#toc)

In [ ]:
def show_TP_perc_plot(df, focal_entity, ylims, figsize):
    
    plt.figure(figsize=figsize)
    g = sns.barplot(data = df, x=focal_entity, y='Name_found', alpha=0.5)
    for index, row in df.iterrows():
        g.text(row.name, 
                   row['Name_found'], 
                   np.round(row['Name_found'], 2), 
                   color='black', 
                   ha="center", 
                   size='large', 
                   verticalalignment='baseline')
    g.set_xlabel('Solutions')
    g.set_ylabel('Percentage of SENSITIVITY (%)')
    g.set(ylim=ylims)
    plt.grid(True, axis='y')
    plt.title('Percentage of Sensitivity (TPR)')
    plt.show()
    
    return None

## 1.5. Function Definition - Metric analysis for phonemes and names <a class="anchor" id="fifth-func-def"></a>

Go to [Table of Contents](#toc)

### Metric analysis for phoneme matching

In [ ]:
def metric_analysis1(maindb, tests):
    
    results = pd.DataFrame()
    
    for test in tests:
        
        actual_name = test[0]
        
        for name in test[1]:
            #print()
            #print('Searched Name:', name, '\nActual Name:', actual_name)

            pn = []
            for t in name.split():
                pn.append(phonetic.DoubleMetaphone().encode(t)[0])

            #print(f"Phoneme for {name} is : {pn}")
            

            db = maindb[maindb.Name.str.contains(actual_name)]
            for row in db.iterrows():


                dist_score = []
                metric0 = lev_ratio(' '.join(row[1]['Phonemes']).lower(), ' '.join(pn).lower())
                dist_score = [max([(lev_ratio(i,j)) for j in row[1]['Phonemes']]) for i in pn]
                metric1 = np.mean(dist_score)
                metric2 = lev_seqratio(row[1]['Phonemes'], pn)
                metric3 = lev_setratio(row[1]['Phonemes'], pn)
                #print([name, , , metric2])


                df2 = {'Name': row[1]['Name'],
                       'Searched': name, 
                       #'Actual': actual_name, 
                       'Phoneme': row[1]['Phonemes'], 
                       'Searched Phoneme': pn, 
                       'Levenshtein Ratio': metric0, 
                       'Custom Levenshtein Ratio': metric1, 
                       'Levenshtein Seq Ratio': metric2, 
                       'Levenshtein Set Ratio': metric3}

                results = results.append(df2, ignore_index = True)


    results.reset_index(drop=True, inplace=True)

    return results



### Metric analysis for name matching

In [ ]:
def metric_analysis2(maindb, tests):
    
    results = pd.DataFrame()
    
    for test in tests:
        
        actual_name = test[0]
        
        for name in test[1]:
            #print()
            #print('Searched Name:', name, '\nActual Name:', actual_name)

            pn = []
            for t in name.split():
                pn.append(phonetic.DoubleMetaphone().encode(t)[0])

            #print(f"Phoneme for {name} is : {pn}")
            

            db = maindb[maindb.Name.str.contains(actual_name)]
            for row in db.iterrows():


                metric0 = lev_ratio(row[1]['Name'].lower(), name.lower())
                metric1 = np.mean([max([(lev_ratio(i,j)) for j in name.lower().split()]) for i in row[1]['Name'].lower().split()])
                metric2 = lev_seqratio(row[1]['Name'].lower(), name.lower())
                metric3 = lev_setratio(row[1]['Name'].lower(), name.lower())


                df2 = {'Name': row[1]['Name'],
                       'Searched': name, 
                       #'Actual': actual_name, 
                       'Phoneme': row[1]['Phonemes'], 
                       'Searched Phoneme': pn, 
                       'Lev Ratio': metric0, 
                       'Custom Lev Ratio': metric1, 
                       'Lev Seq Ratio': metric2, 
                       'Lev Set Ratio': metric3}

                results = results.append(df2, ignore_index = True)


    results.reset_index(drop=True, inplace=True)

    return results



## 1.6. Function Definition - Threshold finalization for phonemes and names metrics <a class="anchor" id="sixth-func-def"></a>

Go to [Table of Contents](#toc)

### Lower Limit using True Positives

In [ ]:
def thres_analysis(maindb, tests):
    
    results = pd.DataFrame()
    
    for test in tests:
        
        actual_name = test[0]
        
        for name in test[1]:
            #print()
            #print('Searched Name:', name, '\nActual Name:', actual_name)

            pn = []
            for t in name.split():
                pn.append(phonetic.DoubleMetaphone().encode(t)[0])

            #print(f"Phoneme for {name} is : {pn}")
            

            db = maindb[maindb.Name.str.contains(actual_name)]
            for row in db.iterrows():


                #metric1 = func[0](row[1]['Name'].lower(), name.lower())
                metric1 = lev_setratio(row[1]['Name'].lower(), name.lower())
                metric2 = np.mean([max([(lev_ratio(i,j)) for j in name.lower().split()]) for i in row[1]['Name'].lower().split()])
                dist_score = []
                #dist_score = [max([(func[1](i,j)) for j in row[1]['Phonemes']]) for i in pn]
                #metric2 = func[1](row[1]['Name'].lower(), name.lower())
                #print([name, lev_seqratio(row[1]['Phonemes'], pn), lev_setratio(row[1]['Phonemes'], pn), metric2])


                df2 = {'Name': row[1]['Name'],
                       'Searched': name, 
                       #'Actual': actual_name, 
                       'Phoneme': row[1]['Phonemes'], 
                       'Searched Phoneme': pn,                         
                       'Lev Set Ratio': metric1, 
                       'Custom Lev Ratio': metric2}

                results = results.append(df2, ignore_index = True)


    results.reset_index(drop=True, inplace=True)

    return results



### Upper Limit using False Positives

In [ ]:
def thres_analysis_unknown(flag, maindb, func, tests):

    perc = np.arange(0,0.9, 0.1)
    perc2 = np.arange(0.9, 1, 0.05)
    perc = np.concatenate([perc, perc2])
    
    results = pd.DataFrame()
    consolidated_results = pd.DataFrame()
    
    start_time = time.time()

    
    name_counter = 1
    for name in tests[-1][1]:

        pn = []
        for t in name.split():
            pn.append(phonetic.DoubleMetaphone().encode(t)[0])

        db = maindb
        print(name_counter, name)
        counter = 0
        for row in db.iterrows():

            metric1 = None
            metric2 = None
            if flag == 1:
                metric1 = func[0](row[1]['Name'].lower(), name.lower())
            elif flag==2:
                dist_score = [max([(func[1](i,j)) for j in row[1]['Phonemes']]) for i in pn]
                metric2 = np.mean(dist_score)
                
            counter += 1
            if counter%5000 == 0:
                print(counter)
            df2 = {'Lev Set Ratio': (metric1), 
                   'Custom Lev Ratio': (metric2)}

            results = results.append(df2, ignore_index = True)
                    
        
        cdf = results.describe(percentiles=perc).T
        display(cdf)
        consolidated_results = consolidated_results.append(cdf, ignore_index = True)
        results = pd.DataFrame()

        name_counter +=1
        
    
    print(f'Execution Time: {time.time()-start_time:,} seconds')
    results.reset_index(drop=True, inplace=True)

    return results, consolidated_results


# 2. Load Data <a class="anchor" id="load-data"></a>

Go to [Table of Contents](#toc)

In [ ]:
names = pd.read_pickle('Final_Names_w_Random.pkl')
names

In [ ]:
names.groupby('List').count()

# 3. Analysis <a class="anchor" id="data-analysis"></a>

Go to [Table of Contents](#toc)

## 3.1 Generate test data <a class="anchor" id="gen-test"></a>

Go to [Table of Contents](#toc)

In [ ]:
names.iloc[np.random.randint(names.shape[0])]

In [ ]:
tests = [["Vladimir Nikolaevich Terentev", ["Vadimir Nikolevi Terente", 
                                            "Valdimi Terente", 
                                            "Baldimi Nikola Terete", 
                                            "Wadimi Terente", 
                                            "Baldimir Derend"]], 
         ["Andrey Alshevskih", ["andre alshveck", 
                                "andrew alshvek", 
                                "andy alshevsik", 
                                "andy alsevsikh", 
                                "Aandre Alshweshk"]],
         ["Andrei Skoch", ["andrew skok", 
                           "andre skokh", 
                           "andrai skosh", 
                           "Aandre kock", 
                           "Andres kok"]], 
         ["Sheikh Aboud Rogo", ["Shake Abud rogo", 
                                "Sheik abod roguo", 
                                "about roguo", 
                                "shaikh rogo", 
                                "Shek Abed Rogo"]], 
         ["Mohammad Fayez Al-Barsha", ["Mohamad Fayis Barsa", 
                                       "Fayes Barhsa", 
                                       "Mohamed Al barsha", 
                                       "Moamad faes albasha", 
                                       "mohamad fayiz"]], 
         ["Wael Mohamad ABED AL RAHMAAN", ["Wel Mohammad ABID RAHMAN", 
                                           "Wael AL-HUSSEINI", 
                                           "Wel Huseni", 
                                           "Mohammad Rehman", 
                                           "Vial Abed"]], 
         ["Khaled Suleiman Fayez ABOU HASSAN", ["Khalid Sulayman Fayiz ABU HASAN", 
                                                "Khaled ABU HASSAN", 
                                                "Kahleed Asan", 
                                                "Kohlid Faye Hasin", 
                                                "Galed Hassan"]], 
         ["David Amos Mazengo", ["Dave Masengo", 
                                 "Daveid Mos Masingo", 
                                 "Dave Masego", 
                                 "Daive Masego", 
                                 "Davie Mos Masigo"]], 
         ["Anas Tals", ["Annas Taals", 
                        "Unus Tuls", 
                        "Aanas Daals", 
                        "Aanas Thaals", 
                        "Aunaas Dhals"]], 
         ["Fares Chihabi", ["Faresh Chiabi", 
                            "Farez Chivi", 
                            "Faresh Jiavi", 
                            "Farsh Jiabi", 
                            "Vares Jabi"]], 
         ["Henni Ziegert", ["Heni Zigart", 
                            "Heini Zegart", 
                            "Heini Cigart", 
                            "Haini Cigat", 
                            "Henny Zigat"]], 
         ["Ebony Macdonald", ["Abony Mcdonald", 
                                 "Albany Macdonall", 
                                 "Ebboni Mactonalt", 
                                 "Ebni Donald", 
                                 "Abby Mctonal"]], 
         ["Joseph Barton", ["Josef Baurtan", 
                            "Josev Boltan", 
                            "Joseph Balton", 
                            "Josh Borton", 
                            "josaif bolton"]], 
         ["Octavia Gracia Crespo Cantero", ["Octave Cresbo Candaro", 
                                            "Octavia Cantaro", 
                                            "Garcia Cantaro", 
                                            "Auktabia Creshpho Cantaro", 
                                            "Ockdawia Garcia"]], 
         ["Kimberly Williams", ["Kimmerly Wiliam", 
                                "Gimmarly William", 
                                "Kimmerli Wilams", 
                                "Kimmaly Villams", 
                                "Kinderli Vilians"]], 
         ["Robert Tapia", ["Rupert Dapia", 
                           "Robby Topia", 
                           "Rob Thapi", 
                           "Robet Daphia", 
                           "Rupat Darpa"]], 
         ["Blanca Chamorro", ["Blanka Chamoro", 
                                  "Blanga Jamoro", 
                                  "Bianca Chmore", 
                                  "Vlanga Jmore", 
                                  "Balanca Janore"]], 
         ["Eliana Ferreras Coello", ["Alina phereras collo", 
                                  "Elaina Goel", 
                                  "Elena fererash Goel", 
                                  "elyana phereros colo", 
                                  "vereros goelo"]], 
         ["Camilla Parini-Correr", ["Camila Perni COuraire", 
                                    "Cammila Gorere", 
                                    "Gamla Barrinni", 
                                    "Camela Goryere", 
                                    "Camiya Barreni Gouryer"]],
         ["Paola Tropea", ["Pola Tropio", 
                          "Baola Tropia", 
                          "Powla Tropica", 
                          "Poula Drobiya", 
                          "Paula Dobia"]], 
         ["Piergiorgio Gentili", ["Pejergio Gentili", 
                                "Pergorgio Jentily", 
                                "Pejerjio Jenely", 
                                "Paijhogio Gently", 
                                "Piyejorgio Jentali"]], 
         ["DUMMY DATA", ["Antonio Hodge", 
                         "Charles Madriz", 
                         "Aleh Vladimir Aziz", 
                         "Luke Powell", 
                         "Anthony Gammell", 
                         "Carlos Vilches", 
                         "Mike Felix", 
                         "Rodrigue Patterson", 
                         "Anya D'Souza", 
                         "Joshua Matthers", 
                         "Michael Myers", 
                         "Owen Price", 
                         "Christopher Peres", 
                         "Mark Gibney", 
                         "Andrew Roy", 
                         "Jared Miller", 
                         "Scott Romero", 
                         "Alex Owen", 
                         "Steve Leben", 
                         "Kirk Jensen", 
                         "Kevin Berry", 
                         "Mike King", 
                         "Mark Bruno", 
                         "Tony Hayes", 
                         "Kevin Sanchez", 
                         "Bill Miller", 
                         "Bob Turner", 
                         "James Wilson", 
                         "Matt Smith", 
                         "Ted Blair", 
                         "Eric Dalton", 
                         "Lynn Davis", 
                         "Dennis Joseph", 
                         "Andrew Ford", 
                         "Douglas Williams", 
                         "Lee Davidson", 
                         "Henry Cohen", 
                         "Carl Jacobs", 
                         "Terry Martinez", 
                         "Joshua Davis"]]]

In [ ]:
# Check if part of name is present in data - Ideally, YES
names[names.Name.str.contains('Davis')]

In [ ]:
# Check if entire name is present - Ideally, NO
for nm in tests[-1][1]:
    print(names[names.Name.str.contains(nm)].shape)

## 3.2. Find the optimal thresholds <a class="anchor" id="find-thresholds"></a>

Go to [Table of Contents](#toc)

### 3.2.1. Find the metrics <a class="anchor" id="find-thresholds-part1"></a>

Go to [Table of Contents](#toc)

### Compare Abydos and python-Levenshtein

In [ ]:
# Abydos - Distance
stat_time = time.time()
for test in tests:
    for i in test[1]:
        (distance.dist_levenshtein(test[0], i))
print(f'Abydos Execution Speed: {np.round((time.time()-stat_time)*1000, 4)} ms')

In [ ]:
# Levenshtein 
stat_time = time.time()
for test in tests:
    for val in test[1]:
        (lev_ratio(test[0], val))
print(f'python-Levenshtein Execution Speed: {np.round((time.time()-stat_time)*1000, 4)} ms')

### Find metric for phoneme matching

In [ ]:
perc = np.arange(0,1, 0.1)
res = metric_analysis1(names, tests)
display(res.describe(percentiles=perc).T)

### Find metric for name matching

In [ ]:
perc = np.arange(0,1, 0.1)

res = metric_analysis2(names, tests)
display(res.describe(percentiles=perc).T)

From the results above, it is visible that the best metrics are as follows:
- Best name comparision metric : **Levenshtein Set Ratio**
- Best phoneme comparision metric : **Custom Levenshtein ratio**

Therefore, using these to finalize the thresholds for the metric...

### 3.2.2. Find the Lower Limit using True Positive data <a class="anchor" id="find-thresholds-part2"></a>

Go to [Table of Contents](#toc)

In [ ]:
res = thres_analysis(names, tests)

#res.sort_values(['Lev Set Ratio'])
#res[res['Lev Set Ratio']<0.75]
#res.sort_values(['Custom Lev Ratio'])
#res[res['Custom Lev Ratio']<60]
perc1 = np.arange(0,0.1, 0.05)
perc2 = np.arange(0.1, 1, 0.1)
perc = np.concatenate([perc1, perc2])
display(res.describe(percentiles=perc).T)

From the observations, to catch the variations, it is best to set the thresholds at 5% as follows:
- Levenshtein Set Ratio = **0.535** *(for name matching)*
- Custom Levenshtein ratio = **0.514** *(for phoneme matching)*

The next stage looks at increasing the limits, for reducing the false positives as much as possible...

### 3.2.3. Find the Upper Limit using False Positive data <a class="anchor" id="find-thresholds-part3"></a>

Go to [Table of Contents](#toc)

In [ ]:
func = [lev_setratio, lev_ratio]
#threshs = [0.368, 0.4]

perc = np.arange(0,1, 0.1)


res1, cres1 = thres_analysis_unknown(1, names, func, tests)
#display(res1.describe(percentiles=perc).T)

In [ ]:
display(cres1.describe(percentiles=perc).T)

In [ ]:
perc = np.arange(0,0.9, 0.1)
perc2 = np.arange(0.9, 1, 0.05)
perc = np.concatenate([perc, perc2])
display(res1.describe(percentiles=perc).T)

In [ ]:
res1.sort_values('Lev Set Ratio')

In [ ]:
# func and threshs defined previously

perc = np.arange(0,1, 0.1)


res2, cres2 = thres_analysis_unknown(2, names, func, tests)
#display(res2.describe(percentiles=perc).T)

In [ ]:
display(cres2.describe(percentiles=perc).T)

In [ ]:
perc = np.arange(0,0.9, 0.1)
perc2 = np.arange(0.9, 1, 0.05)
perc = np.concatenate([perc, perc2])
display(res2.describe(percentiles=perc).T)

In [ ]:
res2.sort_values('Custom Lev Ratio')

In [ ]:
cres1.to_pickle('cres1.pkl')
cres2.to_pickle('cres2.pkl')

In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [ ]:
# Levenshtein Set Ratio

cres11 = cres1.describe().loc['mean':'mean',:]
cres11

In [ ]:
# Custom Levenshtein Ratio

cres22 = cres2.describe().loc['mean':'mean',:]
cres22

In [ ]:
pd.concat([cres22, cres11])

In [ ]:
print(np.mean([0.535, 0.586]))
print(np.mean([0.514, 0.587]))

The earlier thresholds suggestions were kept as:
- Levenshtein Set Ratio = **0.535** *(for name matching)*
- Custom Levenshtein ratio = **0.514** *(for phoneme matching)*

The thresholds using the False positives were chosen at 95%, with values being:
- Levenshtein Set Ratio, *(for name matching)* = **0.586**
- Custom Levenshtein ratio, *(for phoneme matching)* = **0.587**

Therefore, taking average as :
- Levenshtein Set Ratio, *(for name matching)* = **0.56**
- Custom Levenshtein ratio, *(for phoneme matching)* = **0.55**

## 3.3. Finding best solution <a class="anchor" id="find-solution"></a>

Go to [Table of Contents](#toc)

Performance is compared on all the test sets using the finalized metrics **Levenshtein Set Ratio & Custom Levenshtein Ratio** and using the corresponding thresholds of **0.56 & 0.55**. The different solutions are listed below:
1. Using Levenshtein set ratio on the names directly ***(Traditional approach)***
2. Using Levenshtein set ratio on the phonemes of the names
3. Using custom Levenshtein ratio to measure similarity between the phonemes of the names
4. Using BOTH Levenshtein set ratio on names AND custom Levenshtein ratio on phonemes of the names for comparision ***(Proposed approach 1)***
5. Using EITHER Levenshtein set ratio Levenshtein set ratioon names OR custom Levenshtein ratio on phonemes of the names for comparision ***(Proposed approach 2)***

### 3.3.1 Using True Positve names test set

Go to [Table of Contents](#toc)

In [ ]:
funcs = [lev_setratio, 
         lev_setratio, 
         lev_ratio, 
         [lev_setratio, lev_ratio], 
         [lev_setratio, lev_ratio]]


thresholds = [0.56, 0.56, 0.55, [0.56, 0.55], [0.56, 0.55]]

final_results = []

df = pd.DataFrame(columns={'Solution', 'Actual', 'Searched', 'Num_of_matches', 'Name_found', 'Time'})

start_time = time.time()
for i in range(len(tests[:-1])):
    actual_name = tests[i][0]
    display(names[names.Name.str.contains(actual_name)])
    for j in range(len(tests[i][1])):
        searched_name = tests[i][1][j]

        for k in range(5):
            res = solutions(k+1, names, funcs[k], thresholds[k], searched_name, actual_name, False)
            if res is not None:
                final_results.append(res)
        
        df2 = pd.DataFrame({"Solution": [val[0] for val in final_results], 
                            "Actual": actual_name, 
                            "Searched": searched_name, 
                            "Num_of_matches": [val[1].shape[0] for val in final_results],
                            "Name_found": [val[2] for val in final_results], 
                            "Time": [val[3] for val in final_results]})
        
        df = pd.concat([df, df2], ignore_index=True)
        
        
print(f'---- Total Execution Time: {time.time() - start_time} seconds ----')

In [ ]:
df

In [ ]:
df.to_pickle('Temp_save_data_w_Random.pkl')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
import winsound
duration = 3000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

### 3.3.2 Using False Positve names test set

Go to [Table of Contents](#toc)

In [ ]:
funcs = [lev_setratio, 
         lev_setratio, 
         lev_ratio, 
         [lev_setratio, lev_ratio], 
         [lev_setratio, lev_ratio]]


thresholds = [0.56, 0.56, 0.55, [0.56, 0.55], [0.56, 0.55]]

final_results = []

df = pd.DataFrame(columns={'Solution', 'Searched', 'Num_of_matches', 'Time'})

start_time = time.time()
for i in tests[-1][1]:

    searched_name = i

    for k in range(5):
        res = solutions2(k+1, names, funcs[k], thresholds[k], searched_name, False)
        if res is not None:
            final_results.append(res)

    df2 = pd.DataFrame({"Solution": [val[0] for val in final_results], 
                        "Searched": searched_name, 
                        "Num_of_matches": [val[1].shape[0] for val in final_results],
                        "Time": [val[2] for val in final_results]})

    df = pd.concat([df, df2], ignore_index=True)
        
        
print(f'---- Total Execution Time: {time.time() - start_time} seconds ----')

In [ ]:
df.to_pickle('Temp_save_false_w_Random.pkl')

# 4. Generating Performance Plots <a class="anchor" id="gen-plots"></a>

Go to [Table of Contents](#toc)

## 4.1 Using True Positve names test set <a class="anchor" id="sec_4_1"></a>

Go to [Table of Contents](#toc)

In [ ]:
df = pd.read_pickle('Temp_save_data_w_Random.pkl')
df.Num_of_matches = df.Num_of_matches.astype('int64')
df.Name_found = df.Name_found.astype('bool')
df.dtypes


In [ ]:
df

In [ ]:
df2 = df.groupby(['Solution']).mean().reset_index()
df2['Name_found'] = df2['Name_found']*100
df2

In [ ]:
#df22 = df2[~df2.Solution.str.contains('Intermediate')]
show_performance_plots(df2, 'Solution', [(0,40),(0,12000)], "List of solutions", (20,5))
show_TP_perc_plot(df2, 'Solution', (80,105), (20,5))


## 4.2 Using False Positve names test set <a class="anchor" id="sec_4_2"></a>
 
Go to [Table of Contents](#toc)

In [ ]:
df = pd.read_pickle('Temp_save_false_w_Random.pkl')
df.Num_of_matches = df.Num_of_matches.astype('int64')


In [ ]:
df2 = df.groupby(['Solution']).mean().reset_index()
#df2['Name_found'] = df2['Name_found']*100
df2

In [ ]:
show_performance_plots(df2, 'Solution', [(0,40),(0,12000)], "List of solutions", (20,5))

# 5. Finding best solution for names without random names <a class="anchor" id="alt-find-solution"></a>

Go to [Table of Contents](#toc)

Performance is compared on all the test sets using the finalized metrics **Levenshtein Set Ratio & Custom Levenshtein Ratio** and using the corresponding thresholds of **0.56 & 0.55**. The different solutions are listed below:

1. Using Levenshtein set ratio on the names directly ***(Traditional approach)***
2. Using Levenshtein set ratio on the phonemes of the names
3. Using custom Levenshtein ratio to measure similarity between the phonemes of the names
4. Using BOTH Levenshtein set ratio on names AND custom Levenshtein ratio on phonemes of the names for comparision ***(Proposed approach 1)***
5. Using EITHER Levenshtein set ratio Levenshtein set ratioon names OR custom Levenshtein ratio on phonemes of the names for comparision ***(Proposed approach 2)***

## 5.1 Loading data source and test set <a class="anchor" id="sec_5_1"></a>

Go to [Table of Contents](#toc)

In [ ]:
names = pd.read_pickle('Final_Names_wo_Random.pkl')

In [ ]:
tests = [["Vladimir Nikolaevich Terentev", ["Vadimir Nikolevi Terente", 
                                            "Valdimi Terente", 
                                            "Baldimi Nikola Terete", 
                                            "Wadimi Terente", 
                                            "Baldimir Derend"]], 
         ["Andrey Alshevskih", ["andre alshveck", 
                                "andrew alshvek", 
                                "andy alshevsik", 
                                "andy alsevsikh", 
                                "Aandre Alshweshk"]],
         ["Andrei Skoch", ["andrew skok", 
                           "andre skokh", 
                           "andrai skosh", 
                           "Aandre kock", 
                           "Andres kok"]], 
         ["Sheikh Aboud Rogo", ["Shake Abud rogo", 
                                "Sheik abod roguo", 
                                "about roguo", 
                                "shaikh rogo", 
                                "Shek Abed Rogo"]], 
         ["Mohammad Fayez Al-Barsha", ["Mohamad Fayis Barsa", 
                                       "Fayes Barhsa", 
                                       "Mohamed Al barsha", 
                                       "Moamad faes albasha", 
                                       "mohamad fayiz"]], 
         ["Wael Mohamad ABED AL RAHMAAN", ["Wel Mohammad ABID RAHMAN", 
                                           "Wael AL-HUSSEINI", 
                                           "Wel Huseni", 
                                           "Mohammad Rehman", 
                                           "Vial Abed"]], 
         ["Khaled Suleiman Fayez ABOU HASSAN", ["Khalid Sulayman Fayiz ABU HASAN", 
                                                "Khaled ABU HASSAN", 
                                                "Kahleed Asan", 
                                                "Kohlid Faye Hasin", 
                                                "Galed Hassan"]], 
         ["David Amos Mazengo", ["Dave Masengo", 
                                 "Daveid Mos Masingo", 
                                 "Dave Masego", 
                                 "Daive Masego", 
                                 "Davie Mos Masigo"]], 
         ["Anas Tals", ["Annas Taals", 
                        "Unus Tuls", 
                        "Aanas Daals", 
                        "Aanas Thaals", 
                        "Aunaas Dhals"]], 
         ["DUMMY DATA", ["Antonio Hodge", 
                         "Charles Madriz", 
                         "Aleh Vladimir Aziz", 
                         "Luke Powell", 
                         "Anthony Gammell", 
                         "Carlos Vilches", 
                         "Mike Felix", 
                         "Rodrigue Patterson", 
                         "Anya D'Souza", 
                         "Joshua Matthers", 
                         "Michael Myers", 
                         "Owen Price", 
                         "Christopher Peres", 
                         "Mark Gibney", 
                         "Andrew Roy", 
                         "Jared Miller", 
                         "Scott Romero", 
                         "Alex Owen", 
                         "Steve Leben", 
                         "Kirk Jensen", 
                         "Kevin Berry", 
                         "Mike King", 
                         "Mark Bruno", 
                         "Tony Hayes", 
                         "Kevin Sanchez", 
                         "Bill Miller", 
                         "Bob Turner", 
                         "James Wilson", 
                         "Matt Smith", 
                         "Ted Blair", 
                         "Eric Dalton", 
                         "Lynn Davis", 
                         "Dennis Joseph", 
                         "Andrew Ford", 
                         "Douglas Williams", 
                         "Lee Davidson", 
                         "Henry Cohen", 
                         "Carl Jacobs", 
                         "Terry Martinez", 
                         "Joshua Davis"]]]

In [ ]:
for t in tests:
    display(names[names.Name.str.contains(t[0])])

## 5.2 Using True Positve names test set <a class="anchor" id="sec_5_2"></a>

Go to [Table of Contents](#toc)

In [ ]:
funcs = [lev_setratio, 
         lev_setratio, 
         lev_ratio, 
         [lev_setratio, lev_ratio], 
         [lev_setratio, lev_ratio]]


thresholds = [0.56, 0.56, 0.55, [0.56, 0.55], [0.56, 0.55]]

final_results = []

df = pd.DataFrame(columns={'Solution', 'Actual', 'Searched', 'Num_of_matches', 'Name_found', 'Time'})

start_time = time.time()
for i in range(len(tests[:-1])):
    actual_name = tests[i][0]
    display(names[names.Name.str.contains(actual_name)])
    for j in range(len(tests[i][1])):
        searched_name = tests[i][1][j]

        for k in range(5):
            res = solutions(k+1, names, funcs[k], thresholds[k], searched_name, actual_name, False)
            if res is not None:
                final_results.append(res)
        
        df2 = pd.DataFrame({"Solution": [val[0] for val in final_results], 
                            "Actual": actual_name, 
                            "Searched": searched_name, 
                            "Num_of_matches": [val[1].shape[0] for val in final_results],
                            "Name_found": [val[2] for val in final_results], 
                            "Time": [val[3] for val in final_results]})
        
        df = pd.concat([df, df2], ignore_index=True)
        
        
print(f'---- Total Execution Time: {time.time() - start_time} seconds ----')

In [ ]:
df

In [ ]:
df.to_pickle('Temp_save_data_wo_Random.pkl')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

## 5.3 Using False Positve names test set <a class="anchor" id="sec_5_3"></a>

Go to [Table of Contents](#toc)

In [ ]:
funcs = [lev_setratio, 
         lev_setratio, 
         lev_ratio, 
         [lev_setratio, lev_ratio], 
         [lev_setratio, lev_ratio]]


thresholds = [0.56, 0.56, 0.55, [0.56, 0.55], [0.56, 0.55]]

final_results = []

df = pd.DataFrame(columns={'Solution', 'Searched', 'Num_of_matches', 'Time'})

start_time = time.time()
for i in tests[-1][1]:

    searched_name = i

    for k in range(5):
        res = solutions2(k+1, names, funcs[k], thresholds[k], searched_name, False)
        if res is not None:
            final_results.append(res)

    df2 = pd.DataFrame({"Solution": [val[0] for val in final_results], 
                        "Searched": searched_name, 
                        "Num_of_matches": [val[1].shape[0] for val in final_results],
                        "Time": [val[2] for val in final_results]})

    df = pd.concat([df, df2], ignore_index=True)
        
        
print(f'---- Total Execution Time: {time.time() - start_time} seconds ----')

In [ ]:
df.to_pickle('Temp_save_false_wo_Random.pkl')


## 5.4 Generating Performance Plots <a class="anchor" id="sec_5_4"></a>

Go to [Table of Contents](#toc)

### 5.4.1 Using True Positve names test set

In [ ]:
df = pd.read_pickle('Temp_save_data_wo_Random.pkl')
df.Num_of_matches = df.Num_of_matches.astype('int64')
df.Name_found = df.Name_found.astype('bool')
df.dtypes


In [ ]:
df

In [ ]:
df2 = df.groupby(['Solution']).mean().reset_index()
df2['Name_found'] = df2['Name_found']*100
df2

In [ ]:
#df22 = df2[~df2.Solution.str.contains('Intermediate')]
show_performance_plots(df2, 'Solution', [(0,11),(0,3000)], "List of solutions", (20,5))
show_TP_perc_plot(df2, 'Solution', (80,105), (20,5))


### 5.4.2 Using False Positve names test set

In [ ]:
df = pd.read_pickle('Temp_save_false_wo_Random.pkl')
df.Num_of_matches = df.Num_of_matches.astype('int64')


In [ ]:
df2 = df.groupby(['Solution']).mean().reset_index()
#df2['Name_found'] = df2['Name_found']*100
df2


In [ ]:
show_performance_plots(df2, 'Solution', [(0,7),(0,2500)], "List of solutions", (20,5))
